In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader


In [63]:
# Check pyTorch version
torch.__version__

'2.1.2'

In [64]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using: {device}")

Using: cpu


In [65]:
datapath= "/Users/niraj/Desktop/ML-Cognitive-Science/test/Final_data.csv"

In [66]:
df = pd.read_csv(datapath)

In [67]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [68]:
df.head()

,Target,participant,Exp,Condition,item,Sentence,WordNum,Age,TargetPos,Verb,VerbPos,CQ_Type,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,w_9,w_10,w_11,w_12,w_13,w_14,w_15,w_16,w_17,w_18,w_19,w_20,Average_Reading_time,Reading_time_on_target,Response_Recorded.corr,Response_Recorded.rt,Target_Mem,Item_t,Item_p,Item_u,Item_s,Item_c,expName,Item_t_Memory_trials.thisN,Item_t_Memory_trials.thisIndex,Item_t_MemoryTestResp.corr,Item_t_MemoryTestResp.rt,t_Target_Mem_CorResp,t_Target_Mem_CorrectResp,Item_t_Cosine,Item_p_Memory_trials.thisN,Item_p_Memory_trials.thisIndex,Item_p_MemoryTestResp.corr,Item_p_MemoryTestResp.rt,p_Target_Mem_CorResp,p_Target_Mem_CorrectResp,Item_p_Cosine,Item_u_Memory_trials.thisN,Item_u_Memory_trials.thisIndex,Item_u_MemoryTestResp.corr,Item_u_MemoryTestResp.rt,u_Target_Mem_CorResp,u_Target_Mem_CorrectResp,Item_u_Cosine,Item_s_Memory_trials.thisN,Item_s_Memory_trials.thisIndex,Item_s_MemoryTestResp.corr,Item_s_MemoryTestResp.rt,s_Target_Mem_CorResp,s_Target_Mem_CorrectResp,Item_s_Cosine,Item_c_Memory_trials.thisN,Item_c_Memory_trials.thisIndex,Item_c_MemoryTestResp.corr,Item_c_MemoryTestResp.rt,c_Target_Mem_CorResp,c_Target_Mem_CorrectResp,Item_c_Cosine
0,butterfly,A1_E12_Y34,PrePost_Mod,Pre-modified,R15_Aim1_i37,"It was the pretty, shimmering blue butterfly t...",15,Young,7,photographed,11,NaN,2.043582,0.399983,0.566369,0.433314,0.467180,0.516204,0.399971,0.366804,0.533295,0.399932,0.334056,0.283900,0.315596,0.299949,0.250021,NaN,NaN,NaN,NaN,NaN,0.507344,0.399971,0,NaN,NaN,butterfly,clutter,barricade,hummingbird,passenger,Aim1_Final_ToRun_List2,258,36,1,0.782505,lctrl,lctrl,1,34,216,1,3.8052843,rctrl,rctrl,0.088,48,276,1,0.9082908,rctrl,rctrl,0.032,155,156,1,0.867199,rctrl,rctrl,0.487,158,96,1,0.7843946,lctrl,lctrl,NaN
1,mermaid,A1_E12_Y34,PrePost_Mod,Unmodified_Post,R15_Aim1_i16,The folk tale said that it was the mermaid tha...,19,Young,9,trapped,13,Dependency,8.630846,0.516957,0.349751,0.366940,0.283343,0.333261,0.316737,0.299669,0.300390,0.266290,0.300038,0.266773,0.233593,0.232916,0.350182,0.266723,0.283599,0.266592,0.266473,NaN,0.743741,0.300390,1,2.282060,NaN,mermaid,raid,book,fairy,sailor,Aim1_Final_ToRun_List2,26,15,1,0.875974,lctrl,lctrl,1,76,195,0,0.9907946,rctrl,rctrl,0.028,270,255,1,1.6861268,rctrl,rctrl,0.018,286,135,0,2.142522,rctrl,rctrl,0.545,93,75,1,0.8869807,lctrl,lctrl,NaN
2,runner,A1_E12_Y34,PrePost_Mod,Unmodified_Pre,R15_Aim1_i46,I heard that it was the runner that the mentor...,16,Young,7,trained,11,NaN,1.681664,0.400842,0.366426,0.266026,0.283635,0.283253,0.316628,0.300221,0.283160,0.316621,0.333436,0.466473,0.366837,0.283342,0.316617,0.233333,NaN,NaN,NaN,NaN,0.406157,0.316628,0,NaN,NaN,runner,wrens,tower,winner,mentor,Aim1_Final_ToRun_List2,195,45,1,1.088317,lctrl,lctrl,1,49,225,1,0.7259367,rctrl,rctrl,0.032,0,285,1,2.0624101,rctrl,rctrl,0.097,299,165,1,0.754723,rctrl,rctrl,0.593,111,105,1,1.4610028,lctrl,lctrl,NaN
3,sword,A1_E12_Y34,PrePost_Mod,Unmodified_Pre,R15_Aim1_i38,I couldn't believe that it was a sword that th...,16,Young,8,swallowed,12,General,0.313546,0.350083,0.449946,0.450850,0.382726,0.582942,0.300276,0.283303,0.300086,0.366320,0.283605,0.300210,0.283182,0.299796,0.283672,0.399858,NaN,NaN,NaN,NaN,0.351900,0.283303,1,2.732796,NaN,sword,seed,manifold,knife,magician,Aim1_Final_ToRun_List2,66,37,1,0.852112,lctrl,lctrl,1,271,217,0,1.4093314,rctrl,rctrl,0.066,232,277,1,1.0292172,rctrl,rctrl,0.038,54,157,0,1.957806,rctrl,rctrl,0.558,275,97,0,1.060021,lctrl,lctrl,NaN
4,fly,A1_E12_Y34,PrePost_Mod,Unmodified_Post,R15_Aim1_i24,She could not guess that it was the fly that t...,17,Young,9,caught,13,General,0.515149,0.300143,0.299787,0.333482,0.400247,0.482825,0.400239,0.299997,0.266708,0.383099,0.316643,0.283746,0.349857,0.333528,0.349656,0.266792,0.283290,NaN,NaN,NaN,0.345011,0.266708,1,4.563811,NaN,fly,floor,cracker,insect,spider,Aim1_Final_ToRun_List2,160,23,1,1.218718,lctrl,lctrl,1,25,203,0,1.3306159,rctrl,rctrl,0.005,30,263,1,1.1314191,rctrl,rctrl,0.000,277,143,1,3.170142,rctrl,rctrl,0.563,181,

In [69]:
df = df[["Age","Average_Reading_time","Reading_time_on_target","Item_t_MemoryTestResp.rt","Item_t_MemoryTestResp.corr","Item_s_MemoryTestResp.rt","Item_s_MemoryTestResp.corr"]]

In [70]:
df.head()

,Age,Average_Reading_time,Reading_time_on_target,Item_t_MemoryTestResp.rt,Item_t_MemoryTestResp.corr,Item_s_MemoryTestResp.rt,Item_s_MemoryTestResp.corr
0,Young,0.507344,0.399971,0.782505,1,0.867199,1
1,Young,0.743741,0.300390,0.875974,1,2.142522,0
2,Young,0.406157,0.316628,1.088317,1,0.754723,1
3,Young,0.351900,0.283303,0.852112,1,1.957806,0
4,Young,0.345011,0.266708,1.218718,1,3.170142,1


In [71]:
df['Age'] = df['Age'].map({'Young': 0, 'Old': 1})

In [72]:
df.head()

,Age,Average_Reading_time,Reading_time_on_target,Item_t_MemoryTestResp.rt,Item_t_MemoryTestResp.corr,Item_s_MemoryTestResp.rt,Item_s_MemoryTestResp.corr
0,0,0.507344,0.399971,0.782505,1,0.867199,1
1,0,0.743741,0.300390,0.875974,1,2.142522,0
2,0,0.406157,0.316628,1.088317,1,0.754723,1
3,0,0.351900,0.283303,0.852112,1,1.957806,0
4,0,0.345011,0.266708,1.218718,1,3.170142,1


In [73]:
X = df.drop(["Item_t_MemoryTestResp.corr","Item_s_MemoryTestResp.corr"],axis = "columns")
X.head()

,Age,Average_Reading_time,Reading_time_on_target,Item_t_MemoryTestResp.rt,Item_s_MemoryTestResp.rt
0,0,0.507344,0.399971,0.782505,0.867199
1,0,0.743741,0.300390,0.875974,2.142522
2,0,0.406157,0.316628,1.088317,0.754723
3,0,0.351900,0.283303,0.852112,1.957806
4,0,0.345011,0.266708,1.218718,3.170142


In [74]:
# Todo
# remove anything that is higher than standard deviation of 2.5 for each column
# mean+-(standard_deviation* 2.5)

# if this doesn't work try this:
# Median split the participants on their accuracy to comprehension questions (not memory questions)

In [75]:
X["Item_s_MemoryTestResp.rt"].median()

1.06573795

In [76]:

y1 = df["Item_t_MemoryTestResp.corr"]
y2 = df["Item_s_MemoryTestResp.corr"]
print(y1.shape,y2.shape)
y= np.column_stack((y1,y2))
print(y.shape)

(7260,) (7260,)
(7260, 2)


In [80]:
# Splitting the datasets into train and test using 20% as test size

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5808, 5), (1452, 5), (5808, 2), (1452, 2))

## Trying with RandomForest to check the accuracy

In [140]:

rf_clf= RandomForestClassifier()

multi_target_clf = MultiOutputClassifier(estimator = rf_clf)

param_grid = {'estimator__n_estimators': range(1,100)}

# Using GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(multi_target_clf, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Printing the best parameters
print("Best parameters: ", grid_search.best_params_)

# Predicting the test set results
y_pred = grid_search.predict(X_test)

# Calculating the score
score = grid_search.score(X_test,y_test)
print("Score: ", score)

# Printing the Classification report to better analysis
print(classification_report(y_test,y_pred,zero_division=1))


Best parameters:  {'estimator__n_estimators': 71}
Score:  0.46143250688705234
              precision    recall  f1-score   support

           0       0.71      0.87      0.78       984
           1       0.72      0.91      0.81      1036

   micro avg       0.72      0.89      0.79      2020
   macro avg       0.72      0.89      0.79      2020
weighted avg       0.72      0.89      0.79      2020
 samples avg       0.72      0.89      0.78      2020



In [15]:
rf_clf= RandomForestClassifier(n_estimators=99)

# Splitting the datasets into train and test using 20% as test size
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 42)

# print(X_train.shape, X_train.dtype)
# print(y_train.shape, y_train.dtype)
multi_target_clf = MultiOutputClassifier(estimator = rf_clf)

# Fitting the Model with X_train and y_train  Note: here y_train is 2D array 
multi_target_clf.fit(X_train,y_train)

y_pred = multi_target_clf.predict(X_test)
# print(y_pred)
# print(f"This is true one: {y_test}")

# Calculating Cross Validation score to test the performance.
score = multi_target_clf.score(X_test,y_test)
scores = cross_val_score(rf_clf, X, y, cv=5)
print(score)
print(scores)

0.4800275482093664
[0.47107438 0.45867769 0.43526171 0.47451791 0.44490358]


In [92]:
y_test.dtype, y_pred.dtype

(dtype('int64'), dtype('int64'))

In [35]:
# clf_svm =  MultiOutputClassifier(SVC(random_state=42))
# X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 42)
# clf_svm.fit(X_train, y_train)

## Try with SVM model to see if it outperforms RandomForest

In [79]:
from sklearn.multioutput import MultiOutputClassifier

# Creating a base SVM classifier
base_svc = SVC(C = 10, kernel = 'rbf', gamma = 'auto')

# Wrap the SVC model with MultiOutputClassifier
multi_svc = MultiOutputClassifier(base_svc)

# Fit the model to the data
multi_svc.fit(X_train, y_train)

# Evaluate the model on the test set
score = multi_svc.score(X_test, y_test)
print("Score:", score)

Score: 0.4731404958677686


In [74]:
y_pred = multi_svc.predict(X_test[:10])
y_pred.shape

(10, 2)

In [75]:
y_pred[:10]

array([[1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1]])

In [77]:
y_test[:10]

array([[0, 1],
       [0, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [0, 1],
       [1, 1],
       [1, 0],
       [1, 1]])

## Now, Modifying the data such that removing anything that is higher than (mean - standard deviation * 2.5).

In [16]:
df.head()

,Age,Average_Reading_time,Reading_time_on_target,Item_t_MemoryTestResp.rt,Item_t_MemoryTestResp.corr,Item_s_MemoryTestResp.rt,Item_s_MemoryTestResp.corr
0,0,0.507344,0.399971,0.782505,1,0.867199,1
1,0,0.743741,0.300390,0.875974,1,2.142522,0
2,0,0.406157,0.316628,1.088317,1,0.754723,1
3,0,0.351900,0.283303,0.852112,1,1.957806,0
4,0,0.345011,0.266708,1.218718,1,3.170142,1


In [17]:
mean = df["Average_Reading_time"].mean()
sd = df["Average_Reading_time"].std()

In [18]:
value = mean +(2.5* sd)
value

3.8506625966234553

In [19]:
df.shape

(7260, 7)

In [20]:
# Here goes modifying
feature_columns = [col for col in df.columns if col not in ['target1', 'target2']]
def remove_outliers(col):
    mean = np.mean(col)
    SD = np.std(col)
    lower_bound = mean - 2.5 * SD
    upper_bound = mean + 2.5 * SD
    return np.where((col < lower_bound) | (col > upper_bound), np.nan, col)

# Apply the function only to feature columns
df_filtered = df.copy()
df_filtered[feature_columns] = df[feature_columns].apply(remove_outliers)

# If you want to remove the rows with NaN values
df_filtered = df_filtered.dropna()


In [21]:
df_filtered.shape

(7076, 7)

In [22]:
    rf_clf= RandomForestClassifier(n_estimators=99)

    # Splitting the datasets into train and test using 20% as test size
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 42)

    # print(X_train.shape, X_train.dtype)
    # print(y_train.shape, y_train.dtype)
    multi_target_clf = MultiOutputClassifier(estimator = rf_clf)

    # Fitting the Model with X_train and y_train  Note: here y_train is 2D array 
    multi_target_clf.fit(X_train,y_train)
    
    y_pred = multi_target_clf.predict(X_test)
    # print(y_pred)
    # print(f"This is true one: {y_test}")

    # Calculating Cross Validation score to test the performance.
    score = multi_target_clf.score(X_test,y_test)
    scores = cross_val_score(rf_clf, X, y, cv=5)
    print(score)
    print(scores)

0.4696969696969697
[0.46694215 0.4476584  0.434573   0.46694215 0.43939394]


## Checking using Simple Neural Network using PyTorch

In [59]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((5808, 5), (1452, 5), (5808, 2), (1452, 2))

In [81]:

# Convert pandas dataframes to numpy arrays
# X = X.to_numpy() 
# y = y.to_numpy() # this was already a numpy array

# First changing it to numpy array
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

# Convert numpy arrays to PyTorch tensors
X_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(y_train)
X_test = torch.FloatTensor(X_test)
y_test = torch.FloatTensor(y_test)

# Define your dataset
class MyDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        return self.features[index], self.labels[index]

# Create data loaders
batch_size = 16
train_data = MyDataset(X_train, y_train)
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
test_data = MyDataset(X_test, y_test)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)

# Define the model architecture
class MultiOutputClassifier(nn.Module):
    def __init__(self, input_size, output_size):
        super(MultiOutputClassifier, self).__init__()
        self.layer1 = nn.Linear(input_size, 64)
        self.layer2 = nn.Linear(64, 64)
        self.output_layer = nn.Linear(64, output_size)

    def forward(self, x):
        x = nn.functional.relu(self.layer1(x))
        x = nn.functional.relu(self.layer2(x))
        x = self.output_layer(x)
        return x

# Initialize the model
model = MultiOutputClassifier(X_train.shape[1], y_train.shape[1])

# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 250
for epoch in range(num_epochs):
    for i, (features, labels) in enumerate(train_loader):
        outputs = model(features)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

print('Finished Training')

Epoch 10/250, Loss: 0.940854549407959
Epoch 20/250, Loss: 0.7434816360473633
Epoch 30/250, Loss: 0.8639435172080994
Epoch 40/250, Loss: 0.8904860019683838
Epoch 50/250, Loss: 0.8930152058601379
Epoch 60/250, Loss: 0.9516007304191589
Epoch 70/250, Loss: 1.0457701683044434
Epoch 80/250, Loss: 1.1490287780761719
Epoch 90/250, Loss: 0.9410005807876587
Epoch 100/250, Loss: 0.7994152307510376
Epoch 110/250, Loss: 0.8488160371780396
Epoch 120/250, Loss: 0.8706216216087341
Epoch 130/250, Loss: 1.1600459814071655
Epoch 140/250, Loss: 1.0430196523666382
Epoch 150/250, Loss: 0.833387553691864
Epoch 160/250, Loss: 0.9144116640090942
Epoch 170/250, Loss: 0.9208999276161194
Epoch 180/250, Loss: 1.0327879190444946
Epoch 190/250, Loss: 1.0285767316818237
Epoch 200/250, Loss: 1.1585997343063354
Epoch 210/250, Loss: 0.8615858554840088
Epoch 220/250, Loss: 1.0177232027053833
Epoch 230/250, Loss: 0.9146279096603394
Epoch 240/250, Loss: 1.0738551616668701
Epoch 250/250, Loss: 0.7361111640930176
Finished Tr